In [19]:
import pandas as pd
df = pd.read_csv("NIFTY_50_Combined.csv")
df.columns = df.columns.str.strip()
df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")
df = df.sort_values(by="Date").reset_index(drop=True)

#computing daily return
df["Daily Return"] = df["Close"].pct_change()
df = df.dropna().reset_index(drop = True)
expected_return = df["Daily Return"].mean() #expected return

#covariance matrix for risk estimation
cov_matrix = df[["Daily Return"]].cov()

print("Expected Return: ", expected_return)
print("Covariance Matrix: ", cov_matrix)

Expected Return:  0.0006441947666122594
Covariance Matrix:                Daily Return
Daily Return      0.000059


In [20]:
import numpy as np
assets = 7
returns = np.array(df["Daily Return"][:assets])
cov_matrix = df["Daily Return"].iloc[:assets].to_numpy().reshape(-1, 1) @ df["Daily Return"].iloc[:assets].to_numpy().reshape(1, -1)

#setting the penalty factor
penalty = 0.1

#defining the QUBO problem
Q = np.zeros((assets, assets))
#filling the matrix using qubo formulation
for i in range(assets):
    Q[i, i] = -returns[i]+penalty*cov_matrix[i, i]
    for j in range(i+1, assets):
        Q[i, j] = -penalty*cov_matrix[i, j]

#symmetrise the matrix
Q = (Q + Q.T)-np.diag(Q.diagonal())
print(Q)

[[-2.59467933e-03  1.13486826e-06  2.00386066e-06 -6.22607169e-07
   5.19507581e-07 -1.97502626e-06 -4.24006763e-06]
 [ 1.13486826e-06  4.37460545e-03 -3.37613750e-06  1.04897883e-06
  -8.75274942e-07  3.32755682e-06  7.14373588e-06]
 [ 2.00386066e-06 -3.37613750e-06  7.72691770e-03  1.85220390e-06
  -1.54549131e-06  5.87553681e-06  1.26138442e-05]
 [-6.22607169e-07  1.04897883e-06  1.85220390e-06 -2.39835517e-03
   4.80190057e-07 -1.82555175e-06 -3.91916959e-06]
 [ 5.19507581e-07 -8.75274942e-07 -1.54549131e-06  4.80190057e-07
   2.00208444e-03  1.52325257e-06  3.27018129e-06]
 [-1.97502626e-06  3.32755682e-06  5.87553681e-06 -1.82555175e-06
   1.52325257e-06 -7.60406525e-03 -1.24323382e-05]
 [-4.24006763e-06  7.14373588e-06  1.26138442e-05 -3.91916959e-06
   3.27018129e-06 -1.24323382e-05 -1.63104622e-02]]


In [21]:
!pip install mip

In [22]:
from mip import Model, xsum, BINARY, OptimizationStatus
model = Model()

# Original binary variables for asset selection
x = [model.add_var(var_type=BINARY) for _ in range(assets)]
# Additional binary variables to linearize the product for i<j
z = {}
for i in range(assets):
    for j in range(i+1, assets):
        z[(i, j)] = model.add_var(var_type=BINARY)

# Add constraints to enforce z[i,j] = x[i]*x[j] for i < j
for i in range(assets):
    for j in range(i+1, assets):
        model += z[(i,j)] <= x[i]
        model += z[(i,j)] <= x[j]
        model += z[(i,j)] >= x[i] + x[j] - 1

# Build the linearized objective.
# Diagonal terms remain as Q[i,i]*x[i]
objective_expr = xsum(Q[i][i] * x[i] for i in range(assets))
# Off-diagonal terms: for i < j, add  Q[i,j] * z[i,j] plus symmetric term
for i in range(assets):
    for j in range(i+1, assets):
        objective_expr += Q[i][j] * z[(i,j)]

model.objective = objective_expr

# Add budget constraint: select exactly k assets (e.g., k = 3)
k = 3
model += xsum(x[i] for i in range(assets)) == k

model.optimize()

if model.status == OptimizationStatus.OPTIMAL:
    solution = [round(x[i].x) for i in range(assets)]
    print("Optimal asset selection:", solution)
else:
    print("No optimal solution found")


Optimal asset selection: [1, 0, 0, 0, 0, 1, 1]
